In [1]:
pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 2.9 MB/s eta 0:00:00


In [2]:
from praw.models import user
import praw
import pandas as pd
import prawcore.exceptions

In [3]:
# Initialize PRAW with your API credentials
reddit = praw.Reddit(
client_id="nFKOCvQQEIoW2hFeVG6kfA",
client_secret="5BBB4fr-HMPtO8f4jZhle74-fYcDkQ",
user_agent="Icy_Process3191",
)

In [4]:
def get_users_with_post_count(subreddit_name, n, reddit):

    user_post_counts = {}

    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.new(limit=None):
        author = submission.author
        if author:
            author_name = str(author)
            if author_name in user_post_counts:
                user_post_counts[author_name] += 1
            else:
                user_post_counts[author_name] = 1

    users_more_than_n = [user for user, post_count in user_post_counts.items() if post_count > n]

    return users_more_than_n


In [6]:
def get_user_subreddit_metrics(username, reddit):

    subreddit_metrics = {}

    try:
        redditor = reddit.redditor(username)
    except praw.exceptions.APIException as e:
        print(f"Error: {e}")
        return None

    # Iterate through the user's submissions (posts)
    for submission in redditor.submissions.new(limit=None):
        subreddit_name = submission.subreddit.display_name

        if subreddit_name not in subreddit_metrics:
            subreddit_metrics[subreddit_name] = {
                'Number of Posts': 0,
                'Total Karma': 0,
                'Total Comments': 0
            }


        subreddit_metrics[subreddit_name]['Number of Posts'] += 1
        subreddit_metrics[subreddit_name]['Total Karma'] += submission.score
        subreddit_metrics[subreddit_name]['Total Comments'] += submission.num_comments


    df = pd.DataFrame.from_dict(subreddit_metrics, orient='index')
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Subreddit'}, inplace=True)

    return df

In [7]:
get_user_subreddit_metrics("Quiet_Effective7234",reddit)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,Subreddit,Number of Posts,Total Karma,Total Comments
0,marvelstudios,120,2748,1613
1,MCUTheories,44,473,587
2,secretinvasion,15,151,58
3,u_Quiet_Effective7234,1,1,0
4,MarvelTheories,15,148,113
5,MarvelStudiosTheories,1,3,1
6,MarvelStudiosSpoilers,2,2,1


In [19]:
def get_user_metrics(usernames):

    user_metrics = {}

    for username in set(usernames):

        user = reddit.redditor(username)
        user_metrics[username] = {
            'Username': username,
            'Comment Karma': user.comment_karma,
            'Post Karma': user.link_karma,
            'Post Count': sum(1 for _ in user.submissions.new(limit=None)),
            'Comment Count': sum(1 for _ in user.comments.new(limit=None))
        }

    return user_metrics

In [20]:
def get_posts(sub_name, n, top_comments_count):
    subreddit = reddit.subreddit(sub_name)
    top_posts = subreddit.new(limit=n)

    post_data = []
    comment_data = []
    author_usernames = []

    for post in top_posts:
        if not post.is_self:
            continue

        author_username = post.author.name if post.author else None
        author_usernames.append(author_username)

        post_data.append({
            'Author': author_username,
            'Subreddit': post.subreddit.display_name,
            'Post ID': post.id,
            'Score': post.score,
            'Upvotes': post.ups,
            'Downvotes': post.downs,
            'Number of Comments': post.num_comments,
            'Text': post.selftext,
        })

        submission = reddit.submission(id=post.id)
        submission.comment_sort = 'best'
        submission.comments.replace_more(limit=None)

        for comment in submission.comments.list()[:top_comments_count]:
            if hasattr(comment, "author") and comment.author:
                comment_author = comment.author.name
                author_usernames.append(comment_author)
                comment_data.append({
                    'post_id': post.id,
                    'comment_text': comment.body,
                    'comment_score': comment.score,
                    'comment_ups': comment.ups,
                    'comment_downs': comment.downs,
                    'comment_author': comment_author
                })

    user_metrics = get_user_metrics(author_usernames)

    for comment in comment_data:
        username = comment['comment_author']
        comment.update(user_metrics.get(username, {}))

    post_df = pd.DataFrame(post_data)
    comment_df = pd.DataFrame(comment_data)

    return post_df, comment_df

In [21]:
results = get_posts(sub_name='pharmacy', n=10, top_comments_count=5)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [26]:
print(results[0].head())

                 Author Subreddit  Post ID  Score  Upvotes  Downvotes  \
0  Pharmacykilledmysoul  pharmacy  184q9di      8        8          0   
1        Recent-Ad-3884  pharmacy  184o3la     17       17          0   
2     Additional_Ad_110  pharmacy  184noib      2        2          0   
3          carcarcarlll  pharmacy  184lgrj     11       11          0   
4             cowgod180  pharmacy  184j6u8      0        0          0   

   Number of Comments                                               Text  
0                  15  I’ve worked at rite aid and CVS just wondering...  
1                   7  Hi\nI am a pharmacist in community pharmacy an...  
2                   0  Hello my fellow pharmacists, I'm here to get a...  
3                  26  \nI am a retail pharmacist and I just started ...  
4                  31  Needless to say, Incomes in my neck of the woo...  
2


In [23]:
print(results[1].head())

   post_id                                       comment_text  comment_score  \
0  184q9di  Any grocery/big box retailer is 1000x better t...             30   
1  184q9di  better AND in my area albertsons pays signific...             10   
2  184q9di  I completely enjoyed my time with Albertsons f...             11   
3  184q9di  Yes,it’s calmer and metrics aren’t as high. Sa...              6   
4  184q9di  CVS is the worst place for a pharmacist to wor...              3   

   comment_ups  comment_downs   comment_author         Username  \
0           30              0        Will34343        Will34343   
1           10              0  AB-RatedGeneric  AB-RatedGeneric   
2           11              0          chips15          chips15   
3            6              0        Spidahpig        Spidahpig   
4            3              0   VileNonShitter   VileNonShitter   

   Comment Karma  Post Karma  Post Count  Comment Count  
0           1379           6           2            214  


In [ ]:
# Change to Make for Upvotes and Downvotes
estimated_upvotes = round(post.score / (2 * post.upvote_ratio - 1)) if post.upvote_ratio != 0.5 else post.score
estimated_downvotes = estimated_upvotes - post.score
